In [2]:
# Notebook settings
# from tqdm.notebook import trange, tqdm
%alias_magic t timeit
# Settings and imports
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import project_example as pe

# import implicit

Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.


In [3]:
# Load datasets
events = pe.load_data('active1000')
users = pd.read_csv('data/users.csv')
items = pd.read_csv('data/articles.csv')

In [4]:
# Initiate ratings matrix
n_users = len(users)
n_items = len(items)

ratings = np.zeros((n_users, n_items))
print(f'Rating matrix shape: {n_users} x {n_items}')

Rating matrix shape: 1000 x 20344


In [5]:
df_second = events[~events['documentId'].isnull()]
df_second = df_second.drop_duplicates(subset=['userId', 'documentId']).reset_index(drop=True)
df_second = df_second.sort_values(by=['userId', 'time'])

new_user_id = df_second['userId'].values[1:] != df_second['userId'].values[:-1]
new_user_id = np.r_[True, new_user_id]

df_second['uid'] = np.cumsum(new_user_id)
item_ids = df_second['documentId'].unique().tolist()
new_df = pd.DataFrame({'documentId': item_ids, 'tid': range(1, len(item_ids) + 1)})
df_second = pd.merge(df_second, new_df, on='documentId', how='outer')
df_ext = df_second[['uid', 'tid']]

In [6]:
df_third = df_second['activeTime'].fillna(round(df_second['activeTime'].mean(), 0))

In [8]:
df_second['activeTime'].fillna(0, inplace=True)

In [9]:
df_third

0          17.0
1          80.0
2          80.0
3         110.0
4          80.0
          ...  
679350     80.0
679351     80.0
679352    127.0
679353     80.0
679354     80.0
Name: activeTime, Length: 679355, dtype: float64

In [10]:
df_ext.isna().sum()

uid    0
tid    0
dtype: int64

In [98]:
counter = 0
for row in df_ext.itertuples():
    if ratings[row[1]-1, row[2]-1] > 0:
        print('Overwriting!')
        print(ratings[row[1]-1, row[2]-1])
        print('with')
        print(df_second[df_second['uid'] == row[1]]['activeTime'].values[0])
    ratings[row[1]-1, row[2]-1] = df_second[df_second['uid'] == row[1]]['activeTime'].values[0]

ratings

array([[ 17.,  17.,  17., ...,   0.,   0.,   0.],
       [  0.,   0., 435., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  nan,   0.,   0.],
       [  0.,   0.,  nan, ...,   0.,  nan,  nan],
       [ 68.,   0.,  68., ...,   0.,   0.,   0.]])

In [99]:
ratings.shape

(1000, 20344)